In [1]:
from keras.preprocessing import sequence
from keras.datasets import imdb
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger
import tensorflow as tf




Using TensorFlow backend.


In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history[metric])
  plt.plot(history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [2]:
vocabulary_size = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocabulary_size)
print('Loaded dataset with {} training samples, {} test samples'
      .format(len(X_train), len(X_test)))


Loaded dataset with 25000 training samples, 25000 test samples


In [3]:
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}

In [4]:
print('Maximum review length: {}'.format(
len(max((X_train + X_test), key=len))))

Maximum review length: 2697


In [5]:
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

In [6]:
# is TextVectorization needed ?

#Create the layer, and pass the dataset's text to the layer's .adapt method:
#vectorizer = TextVectorization(max_tokens=vocabulary_size, output_mode="int")
#vectorizer.adapt(X_train)

In [8]:
embedding_size=32

embedding_size=32
model=Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(Bidirectional(LSTM(100,  return_sequences=True)))
model.add(Bidirectional(LSTM(50,  return_sequences=True)))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 500, 200)          106400    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 500, 100)          100400    
_________________________________________________________________
dense_1 (Dense)              (None, 500, 1)            101       
Total params: 366,901
Trainable params: 366,901
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

In [10]:
# save best parameter estimate using val loss.
checkpointer = ModelCheckpoint("best_base_model.tf", monitor="val_loss", mode="auto", save_best_only = True, verbose=1)
# early stopping.
earlystoper = EarlyStopping(monitor = 'val_loss', patience = 2, verbose = 1) # patience is num. epochs wo improvement.

# save the training history.
csv_logger = CSVLogger('trainingBaseModel.log', separator=',', append=False) 
# compile the callbacks together
callbackser = [earlystoper, checkpointer, csv_logger]

In [ ]:
batch_size = 64
num_epochs = 3
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]
model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), 
          batch_size=batch_size, epochs=num_epochs)

Epoch 1/3
231/390 [================>.............] - ETA: 2:06 - loss: 0.6385 - accuracy: 0.6327

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', scores[1])

In [ ]:
history = pd.read_csv('trainingBaseModel.log', sep=',', engine='python')